## Hopper Cutter Comparison
This notebook elaborates a comparison between two cases of a single run:
* First case: Trailing suction hopper dredger
    * The subprocesses are defined as follows
        * Sailing empty (MoveActivity)
        * Loading (ShiftAmountActivity)
        * Sailing Full (MoveActivity)
        * Unloading (ShiftAmountActivity)
        * Simulating for Hopper (BasicActivity for Vessel1)
    * A sequential activity is defined which is made up of the subprocesses (shows a sequence of different subprocesses)
    * A While activity is 


* Second case: Cutter suction dredger + barges
    * The sub


### 0. Import Libraries

In [ ]:
import datetime, time
import simpy

import shapely.geometry
import pandas as pd
import numpy as np
import itertools as it

import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

### 1. Define Object Classes
Object classes including Site object and TransportProcessingResource object are defined to show the dredging site and transportation process of dredger material respectively

In [ ]:
# create a Site object based on desired mixin classes
Site = type(
    "Site",
    (
        core.Identifiable,
        core.Log,
        core.Locatable,
        core.HasContainer,
        core.HasResource,
    ),
    {},
)

# create a TransportProcessingResource object based on desired mixin classes
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,
        core.Log,
        core.ContainerDependentMovable,
        core.Processor,
        core.HasResource,
    ),
    {},
)

### 2. Create Objects (Site Objects and Vessel Objects) and Define the Run Function

In [ ]:
def run(NR_BARGES, NR_HOPPERS, total_amount):
    simulation_start = 0
    my_env = simpy.Environment(initial_time=simulation_start)
    registry = {}

    
    
    
    #create site objects
    location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)
    data_from_site = {"env": my_env,
                      "name": "from_site",
                      "geometry": location_from_site,
                      "capacity": total_amount,
                      "level": total_amount,
                      "nr_resources":1}
    from_site = Site(**data_from_site)

    location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)
    data_to_site = {"env": my_env,
                    "name": "to_site",
                    "geometry": location_to_site,
                    "capacity": total_amount,
                    "level": 0,
                    "nr_resources":7}
    to_site = Site(**data_to_site)
    
    #create vessel objects
    vessels = {}
    
    for i in range(NR_BARGES):
        vessels[f"barge{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"barge_{i}",
            geometry=location_from_site,
            capacity=20,
            compute_v=lambda x: 10
        )
        
    cutter = TransportProcessingResource(
        env=my_env,
        name=f"cutter",
        geometry=location_from_site,
        capacity=10,
        compute_v=lambda x: 10
    )
    vessels['cutter'] = cutter
    
    for i in range (NR_HOPPERS):
        vessels[f"hopper{i}"] = TransportProcessingResource(
            env=my_env,
            name=f"hopper_{i}",
            geometry=location_from_site,
            capacity=5,
            compute_v=lambda x: 10
        )
    hopper = TransportProcessingResource(
        env=my_env,
        name=f"hopper",
        geometry=location_from_site,
        capacity=5,
        compute_v=lambda x: 10
    )
    
    vessels['hopper'] = hopper
       
    
    
    
    
    #Create activity objects
    activities_cutter = {}
    activities_hopper = {}
    
    for i in range(NR_BARGES):
        requested_resources={}
        
        amount = np.random.randint(4,6) #loading
        duration = np.random.randint(2000,3000) #sailing and unloading
               
        a= [
            model.BasicActivity(
                        env=my_env,
                        name=f"basic activity:"+vessels[f"barge{i}"].name,
                        registry=registry,
                        duration=duration,
                        additional_logs=[vessels[f"barge{i}"]],
                    ),
        model.MoveActivity(
                        env=my_env,
                        name=f"sailing empty:"+vessels[f"barge{i}"].name,
                        registry=registry,
                        mover=vessels[f"barge{i}"],
                        destination=from_site,
                        duration=duration,
                    ),
        model.ShiftAmountActivity(
                        env=my_env,
                        name=f"loading:"+vessels[f"barge{i}"].name,
                        registry=registry,
                        processor=cutter,
                        origin=from_site,
                        destination=vessels[f"barge{i}"],
                        amount=amount,
                        duration=500*amount,
                        requested_resources=requested_resources,
                    ),
        model.MoveActivity(
                        env=my_env,
                        name=f"sailing full:"+vessels[f"barge{i}"].name,
                        registry=registry,
                        mover=vessels[f"barge{i}"],
                        destination=to_site,
                        duration=duration,
                    ),
        model.ShiftAmountActivity(
                        env=my_env,
                        name=f"unloading:"+vessels[f"barge{i}"].name,
                        registry=registry,
                        processor=vessels[f"barge{i}"],
                        origin=vessels[f"barge{i}"],
                        destination=to_site,
                        amount=amount,
                        duration=duration,
                        requested_resources=requested_resources,
                    )
        ]
            
        
        activities_cutter[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle_cutter{i}",
            registry=registry,
            sub_processes=[model.SequentialActivity(
                env=my_env,
                name=f"sequential_activity_subcycle_cutter{i}",
                registry=registry,
                sub_processes= a
            )],
            condition_event=[{"type": "container",
                              "concept": to_site,
                              "state": "full",
                              "id_":"default_reservations"}]
        )

    for i in range (NR_HOPPERS):
        requested_resources={}      
        
        b= [
            model.BasicActivity(
                        env=my_env,
                        name=f"basic activity:"+vessels[f"hopper{i}"].name,
                        registry=registry,
                        duration=duration,
                        additional_logs=[vessels[f"hopper{i}"]],
                    ),
        model.MoveActivity(
                        env=my_env,
                        name=f"sailing empty:"+vessels[f"hopper{i}"].name,
                        registry=registry,
                        mover=vessels[f"hopper{i}"],
                        destination=from_site,
                        duration=duration,
                    ),
        model.ShiftAmountActivity(
                        env=my_env,
                        name=f"loading:"+vessels[f"hopper{i}"].name,
                        registry=registry,
                        processor=hopper,
                        origin=from_site,
                        destination=vessels[f"hopper{i}"],
                        amount=amount,
                        duration=200*amount,
                        requested_resources=requested_resources,
                    ),
        model.MoveActivity(
                        env=my_env,
                        name=f"sailing full:"+vessels[f"hopper{i}"].name,
                        registry=registry,
                        mover=vessels[f"hopper{i}"],
                        destination=to_site,
                        duration=duration,
                    ),
        model.ShiftAmountActivity(
                        env=my_env,
                        name=f"unloading:"+vessels[f"hopper{i}"].name,
                        registry=registry,
                        processor=vessels[f"hopper{i}"],
                        origin=vessels[f"hopper{i}"],
                        destination=to_site,
                        amount=amount,
                        duration=duration,
                        requested_resources=requested_resources,
                    )
        ]
        
    
        
        activities_hopper[f"activity{i}"] = model.WhileActivity(
            env=my_env,
            name=f"while_sequential_activity_subcycle_hopper{i}",
            registry=registry,
            sub_processes=[model.SequentialActivity(
                env=my_env,
                name=f"sequential_activity_subcycle_hopper{i}",
                registry=registry,
                sub_processes= b
            )],
             condition_event=[{"type": "container",
                              "concept": to_site,
                              "state": "full",
                              "id_":"default_reservations"}
                             ]
        )
        
                           
    model.register_processes([*activities_hopper.values(), *activities_cutter.values()])
    my_env.run()
    
    return {
        "vessels": vessels,
        "activities_hopper":activities_hopper,
        "activities_cutter":activities_cutter,
        "from_site":from_site,
        "to_site":to_site,
    }


In [ ]:
%%time

res = run(5,10,3000)
vessels = res['vessels']
activities_cutter = res['activities_cutter']
activities_hopper = res['activities_hopper']
cutter = vessels['cutter']
hopper = vessels['hopper']
to_site = res['to_site']
from_site = res['from_site']

In [ ]:
fig = plot.get_gantt_chart([*vessels.values()], id_map=[*activities_hopper.values(), *activities_cutter.values()])
#fig = plot.get_gantt_chart([*vessels.values()], id_map=[activities_hopper[x] for x in activities_hopper])

In [ ]:
fig = plot.get_gantt_chart(
    [from_site, to_site, hopper],
    id_map=[*activities_hopper.values(), *activities_cutter.values()]
)

In [ ]:
def expand(a, namespace):
    n = len(namespace)
    sa = []
    if a.name[0:n]==namespace:
        sa.append(a)
    if hasattr(a, "sub_processes"):
        for s in a.sub_processes:
            if s.name[0:n]==namespace:
                sa.append(s)
            sa+=expand(s,namespace)
    return sa
all_activities_hopper = []
for a in [*activities_hopper]:
    all_activities_hopper+=expand(activities_hopper[a],'load')
all_activities_hopper
fig = plot.get_gantt_chart(all_activities_hopper)

In [ ]:
fig = plot.get_step_chart([from_site, to_site])